In [1]:
REPLAYS_PATH = '/home/kvu/gfootball-replays/1500/working'

In [2]:
import os
import json
import math

os.listdir(REPLAYS_PATH)

['4349081.json',
 '4660493.json',
 '4333798.json',
 '4157921.json',
 '4527938_info.json',
 '4687942.json',
 '4582418.json',
 '4330804.json',
 '4411430.json',
 '4608748_info.json',
 '4549196_info.json',
 '4545904.json',
 '4379922_info.json',
 '4458015.json',
 '4539963_info.json',
 '4112418.json',
 '4549960.json',
 '4075882_info.json',
 '4611009.json',
 '4495452.json',
 '4564724_info.json',
 '4318317_info.json',
 '4632527_info.json',
 '4560366_info.json',
 '3871261.json',
 '4373614.json',
 '4449413.json',
 '4452329.json',
 '3918881.json',
 '4655594_info.json',
 '4343342_info.json',
 '4060614_info.json',
 '4600045_info.json',
 '4402107_info.json',
 '4622425.json',
 '4606241.json',
 '4683534_info.json',
 '4940950.json',
 '4328717.json',
 '4804443.json',
 '4710848.json',
 '4793122.json',
 '4705866_info.json',
 '4611775_info.json',
 '4539106.json',
 '4354413_info.json',
 '4131208.json',
 '4677633_info.json',
 '4181189.json',
 '4571106.json',
 '4278767_info.json',
 '4710755_info.json',
 '4555

In [5]:
len(os.listdir(REPLAYS_PATH))

844

In [4]:
for replay in os.listdir(REPLAYS_PATH):
    if 'info' in replay:
        os.remove(os.path.join(REPLAYS_PATH, replay))

In [6]:
def get_distance(pos1,pos2):
    return ((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2)**0.5

def get_heading(pos1,pos2):
    raw_head=math.atan2(pos1[0]-pos2[0],pos1[1]-pos2[1])/math.pi*180

    if raw_head<0:
        head=360+raw_head
    else:
        head=raw_head
    return head

In [29]:
EntireReplays = []

CENTER_X = 0
CENTER_Y = 0
SIDELINE_X = 0
SIDELINE_Y = 0.42

for game in os.listdir(REPLAYS_PATH):
    GameFullPath = os.path.join(REPLAYS_PATH, game)
    f = open(GameFullPath)
    GameJson = json.load(f)
    if None in GameJson['rewards']:
        continue
    if GameJson['rewards'][0] < GameJson['rewards'][1]:
        continue
    WINNER = 0
#     WINNER = 0 if GameJson['rewards'][0] > GameJson['rewards'][1] else 1
    for step in GameJson['steps']:
        Obs = step[WINNER]['observation']['players_raw'][0]
        Action = step[WINNER]['action']
        if Obs['ball_owned_player'] != 0:
            continue
        DfRow = {}
        LeftTeamPos = Obs['left_team']
        RightTeamPos = Obs['right_team']
        CurPlayerPos = Obs['left_team'][Obs['active']]

        DfRow['curx'] = get_distance(CurPlayerPos, (CENTER_X, CENTER_Y))
        DfRow['cury'] = get_distance(CurPlayerPos, (SIDELINE_X, SIDELINE_Y))

        PlayerNum = 0
        for player in LeftTeamPos:
            PlayerRelativeDist = get_distance(CurPlayerPos, player)
            PlayerHeadingAngle = get_heading(CurPlayerPos, player)
            NameDist = 'l' + str(PlayerNum) + 'd'
            NameAngle = 'l' + str(PlayerNum) + 'a'
            DfRow[NameDist] = PlayerRelativeDist
            DfRow[NameAngle] = PlayerHeadingAngle
            PlayerNum += 1

        PlayerNum = 0
        for player in RightTeamPos:
            PlayerRelativeDist = get_distance(CurPlayerPos, player)
            PlayerHeadingAngle = get_heading(CurPlayerPos, player)
            NameDist = 'r' + str(PlayerNum) + 'd'
            NameAngle = 'r' + str(PlayerNum) + 'a'
            DfRow[NameDist] = PlayerRelativeDist
            DfRow[NameAngle] = PlayerHeadingAngle
            PlayerNum += 1


        DfRow['action'] = Action[0]
        EntireReplays.append(DfRow)

In [30]:
len(EntireReplays)

26848

In [31]:
import pandas as pd

df = pd.DataFrame(EntireReplays)

In [32]:
df

,curx,cury,l0d,l0a,l1d,l1a,l2d,l2a,l3d,l3a,...,r6a,r7d,r7a,r8d,r8a,r9d,r9a,r10d,r10a,action
0,0.904952,1.027798,0.019659,30.040492,0.314671,300.293509,0.000000,0.000000,0.047644,202.170086,...,240.150338,0.227017,263.810601,0.055515,226.729180,0.033951,326.832035,0.242887,307.352755,2
1,0.914452,1.036358,0.018878,358.640224,0.322679,299.372869,0.000000,0.000000,0.052207,211.478414,...,241.495944,0.231226,263.722075,0.058916,230.598006,0.036111,321.429999,0.250318,305.792693,9
2,0.777533,0.874441,0.121532,71.251613,0.338516,310.715160,0.068873,268.781580,0.000000,0.000000,...,230.640312,0.237865,284.599680,0.261432,223.760928,0.148098,296.791930,0.266447,333.921139,4
3,0.788621,0.886248,0.103669,69.218193,0.337702,310.767991,0.070392,265.716728,0.000000,0.000000,...,234.528356,0.249799,283.901090,0.279788,224.705063,0.160656,291.930567,0.273405,330.409734,13
4,0.799943,0.898360,0.088570,67.872401,0.336691,310.799802,0.072790,262.842325,0.000000,0.000000,...,236.999294,0.262034,283.199245,0.298688,225.413122,0.175643,287.482037,0.281927,326.980152,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26843,0.888234,0.952481,1.711920,87.677355,0.844068,71.741151,0.811259,82.272316,1.151942,88.571942,...,87.529999,0.550130,73.830063,0.662525,110.349605,1.125774,88.391570,0.822526,66.926249,4
26844,0.893245,0.958719,1.717071,87.803487,0.842613,71.869813,0.811385,82.373400,1.157031,88.753705,...,87.628231,0.546092,73.933722,0.664641,110.786905,1.129104,88.557457,0.820699,67.062792,4
26845,0.923747,1.020036,0.052405,122.124038,0.349701,305.346224,0.051566,354.259961,0.000000,0.000000,...,277.461786,0.219003,296.644168,0.236434,199.620542,0.061439,331.839660,0.258671,325.428690,7
26846,0.927697,1.019996,0.045906,125.889601,0.359134,307.138258,0.054416,348.316154,0.000000,0.000000,...,280.755068,0.223829,297.765012,0.234069,202.011661,0.064552,328.825969,0.268214,325.786977,6


In [33]:
df.to_csv('replays.csv', sep=',')